# Franken-Llama

This project aims to analyze the architecture of current SoTA transformer models, in particular Llama, by examining how the output and feature maps change by removing and/or repeating certain attention blocks of the model.

Ultimately, we want to explore ways of enhancing their efficiency for potential deployment on micro-controller units.

In [1]:
## WARNING: ask hugging face for llama2 access first!
# from huggingface_hub import notebook_login
# notebook_login()

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import gc
import importlib # debugging purposes

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


We are also going to import the custom libraries made specifically for this project.

In [3]:
import llama_utils
import modified_llama
import visualizer
importlib.reload(visualizer)

visualizer.set_display_mode(False)

False

## LLama2
LLaMA 2 (Large Language Model Meta AI 2) is Meta's open-source family of advanced language models designed for tasks like text generation, summarization, and question answering. 

It improves upon the original LLaMA with enhanced training stability, fine-tuning, and safety features, making it more effective and accessible for research and commercial use. Available in different sizes, LLaMA 2 is optimized for both efficiency and scalability, allowing deployment on various hardware configurations. 

### Testing Llama2
Just for fun, let's briefly test Llama2 capabilities by asking it to write a continuation to the string "Once upon a time".

We'll also retrieve both the attention maps and the hidden states, which we'll use later in the visualization step.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
import torch
from transformers.modeling_outputs import CausalLMOutput

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="eager")


In [5]:
# model.to(device)

# model.config.output_attentions = True
# model.config.return_dict = True
# model.config.output_hidden_states = True

# input_text = "Once upon a time"
# input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

# outputs = model.generate(input_ids, max_length=50, output_attentions=True, return_dict_in_generate=True, output_hidden_states=True)

# generated_ids = outputs.sequences
# attention_weights = outputs.attentions
# states = outputs.hidden_states

# generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
# print(generated_text)

In [6]:
import modified_llama
mod_model = modified_llama.ModifiedLlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="eager")
mod_model.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ModifiedLlamaForCausalLM(
  (model): ModifiedLlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
 

In [7]:
importlib.reload(modified_llama)

mod_model.config.output_attentions = True
mod_model.config.return_dict = True
mod_model.config.output_hidden_states = True

input_text = "Once upon a time"
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

outputs = mod_model.generate(input_ids, max_length=50, output_attentions=True, return_dict_in_generate=True, output_hidden_states=True)

generated_ids = outputs.sequences
attention_weights = outputs.attentions
states = outputs.hidden_states

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH
AAAAH


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Once upon a time, in a small village nestled in the rolling hills of Tuscany, there was a young girl named Sophia. She lived with her parents and younger brother in a small stone cottage on the outskir


In [9]:
out = mod_model(input_ids)

AAAAH


In [11]:
out

CausalLMOutputWithPast(loss=None, logits=tensor([[[  0.1041,  -0.2213,   0.3135,  ...,   1.3281,   1.8789,   0.6436],
         [-10.0312,  -3.3457,  -2.4512,  ...,  -6.4023,  -7.8711,  -7.7891],
         [ -0.4927,   3.3691,   6.6250,  ...,   0.3577,  -2.4668,   0.6431],
         [ -2.0449,   1.1348,   6.9961,  ...,   0.7515,  -2.7344,  -0.0752],
         [  0.7505,   2.2188,   6.5039,  ...,   0.8848,   1.4072,  -1.9160]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.4199,  0.0054,  0.0629,  ...,  0.1068, -0.0737, -0.0549],
          [-0.6455,  0.3611,  0.0117,  ...,  0.5156, -0.2974,  0.5708],
          [-0.4707,  0.2583,  0.4360,  ...,  0.5620, -0.4036,  0.5762],
          [ 0.2075,  0.2979, -0.7393,  ..., -0.3325,  0.3352, -0.3381],
          [-0.1710,  0.3140, -0.3394,  ...,  0.0321,  0.0871, -0.0385]],

         [[ 1.2744,  1.0127, -0.3867,  ...,  0.5376, -0.2581,  0.5215],
          [ 0.6162,  0.6699, -0.2593,  ...,

Llama2 has 32 decode layers, each one of them representing a block in a transformer decoder.

Each LlamaDecoderLayer is made of an Attention block, a MLP and 2 RMSNorms.

### Visualizing attention maps
#### Layer 1
We're first going to visualize the first layer attention map, and we'll go deeper.
We're also just going to visualize the first head attention map, and as such we can expect the first token to get more focus compared to the others.

In [6]:
layer_idx = 0
head_idx = 0

tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
visualizer.plot_attention_map(outputs.attentions, tokens, layer_idx=layer_idx, head_idx=head_idx)

For the first layer, we can see how the model outputs a `SEQ_LENxSEQ_LEN` attention map. It's interesting how some the last tokens have some importance w.r.t to themselves.

In [7]:
layer_idx = 20
head_idx = 0

tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())
visualizer.plot_attention_map(outputs.attentions, tokens, layer_idx=layer_idx, head_idx=head_idx)

Analyzing the tokens later becomes much more difficult, as the attention is computed only w.r.t. the final token of the generated sequence.

This is because Llama2, just like many other decoder only transformers, uses Masked Causal attention, which generates attention only for the last token in order to speed up computation.

Going deeper into the net, we can see how the attention map increases with a value of `layer_depth + seq_len`.

### ...finally, Franken-LLama!
There are many ways to achieve the goal of running Llama2 without some running some specific layers (or repeating the same layer multiple times).

For this reason, I've developed two ways of achieving our goal, which vary in complexity/attainment to the original Llama2 specification.

#### Using the from-the-ground-up classes

For the scope of this project, I've decided to develop 2 classes which allow us to use a single attention layer without doing the background operations that Llama2 does in its backend (such as computing a rotary embedding layer).

For this reason, the quality of the output is much lower, however the computation is faster.


In [39]:
import llama_utils
importlib.reload(llama_utils)
test_model = llama_utils.SimpleLlamaSkipRepeat(mod_model, target_layers_idx=[], num_repeats=1, config=mod_model.config, skip_all=False)

c:\Users\ASUS\Desktop\APAI-pw\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(
c:\Users\ASUS\Desktop\APAI-pw\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [40]:
output_res = test_model.generate(input_ids, max_length=20)
generated_ids = output_res.sequences
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\ASUS\Desktop\APAI-pw\.venv\Lib\site-packages\transformers\generation\utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on <attribute 'type' of 'torch.device' objects>. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('<attribute 'type' of 'torch.device' objects>') before running `.generate()`.
  warnings.warn(


Once upon a time ...



 Unterscheidung ... upon a time ... upon ... upon ...


In [42]:
output_res = test_model(input_ids, test_output=True)
token_ids = output_res.squeeze().tolist()
generated_text = tokenizer.decode(token_ids, skip_special_tokens=True)

In [43]:
print(output_res)
print(generated_text)

# TODO: find a way to increase output length at each time

tensor([[   13,  2501,   263, 23196,  2023]], device='cuda:0')

 upon a nobody ...


#### Using the modified transformer classes

Like I mentioned, many of the operations required for Llama to work properly have been skipped in the previous version of the classes. The modified transformer classes represent an attempt to circumvent this issues by taking the source code of Llama2 and modifying it to skip or repeat some layers.


### LLama Extraction of attention layers